# Final
Feature Engineering is the key.

  
reference:  
https://www.kaggle.com/mjbahmani/statistical-analysis-for-elo  
https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/82055  
https://www.kaggle.com/chauhuynh/my-first-kernel-3-699  
https://www.kaggle.com/fabiendaniel/elo-world  
https://www.kaggle.com/raddar/target-true-meaning-revealed  
https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/82036#479038  
https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/  
https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering this is really helpfull  
https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

lightgbm params tuning:
https://www.kaggle.com/garethjns/microsoft-lightgbm-with-parameter-tuning-0-823

datetime feature  
https://datascience.stackexchange.com/questions/27273/does-it-make-sense-that-datetime-encodes-one-hot-vector-like-one-hot-encoding-or

### Plan
After the analysis of mid-term project, we achieved a performance (RMSE) at 3.6524 on validation dataset using lightGBM model. Now we need to think can we do better on it? So we are planning to conduct the analysis further:
1. Can we get better score by using different or more features?
2. Can we get better score by optimizing the model?
3. Can we get better score by dealing with outliers more properly?
4. we need to do CV.
5. Xgboost/Catboost.


In [1]:
import pandas as pd
import os
import numpy as np
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
# import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

print(os.listdir("../data"))

['Data_Dictionary.xlsx', 'new_merchant_transactions.csv', 'test.csv', '~$Data_Dictionary.xlsx', 'merchants.csv', 'historical_transactions.csv', 'train.csv', '.ipynb_checkpoints', 'sample_submission.csv']


### Load Data

In [2]:
train_raw = pd.read_csv('../data/train.csv')
test_raw = pd.read_csv('../data/test.csv')
merchants_raw = pd.read_csv('../data/merchants.csv')
historical_transactions_raw = pd.read_csv('../data/historical_transactions.csv')
new_transactions_raw = pd.read_csv('../data/new_merchant_transactions.csv')
print('...loaded')

...loaded


In [3]:
# assign to new dataframe, save the raw data to be able to reuse again
train = train_raw
train.name = 'train data'
test = test_raw
test.name = 'test data'
historical_transactions = historical_transactions_raw
historical_transactions.name = 'historical transaction data'
new_transactions = new_transactions_raw
new_transactions.name = 'new transaction data'
merchants = merchants_raw
merchants.name = 'merchants data'

### Fill Null Data

In [4]:
# view NA data
print("train set NA data {}".format(train.isna().sum()))
print("test set NA data {}".format(test.isna().sum()))
print("new transaction set NA data {}".format(new_transactions.isna().sum()))
print("historical transaction set NA data {}".format(historical_transactions.isna().sum()))
print("merchants data set NA data {}".format(merchants.isna().sum()))

train set NA data first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64
test set NA data first_active_month    1
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64
new transaction set NA data authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3               55922
merchant_category_id         0
merchant_id              26216
month_lag                    0
purchase_amount              0
purchase_date                0
category_2              111745
state_id                     0
subsector_id                 0
dtype: int64
historical transaction set NA data authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
categ

In [5]:
# helper function to fill na data with random value in the same column with on-NaN data
from random import choices

def randomiseMissingData(df):
    "randomise missing data for DataFrame (within a column)"
    for col in df.columns:
        data = df[col]
        mask = data.isna()
        samples = choices( data[~mask].values , k = mask.sum() )
        data[mask] = samples

In [6]:
# fill the NA data, this will take some time

for df in [historical_transactions, new_transactions, merchants]:
    print('fill NA data in {}'.format(df.name))
    randomiseMissingData(df)
print('...done')

fill NA data in historical transaction data
fill NA data in new transaction data
fill NA data in merchants data
...done


In [7]:
# view NA data again
print("train set NA data {}".format(train.isna().sum()))
print("test set NA data {}".format(test.isna().sum()))
print("new transaction set NA data {}".format(new_transactions.isna().sum()))
print("historical transaction set NA data {}".format(historical_transactions.isna().sum()))
print("merchants data set NA data {}".format(merchants.isna().sum()))

train set NA data first_active_month    0
card_id               0
feature_1             0
feature_2             0
feature_3             0
target                0
dtype: int64
test set NA data first_active_month    1
card_id               0
feature_1             0
feature_2             0
feature_3             0
dtype: int64
new transaction set NA data authorized_flag         0
card_id                 0
city_id                 0
category_1              0
installments            0
category_3              0
merchant_category_id    0
merchant_id             0
month_lag               0
purchase_amount         0
purchase_date           0
category_2              0
state_id                0
subsector_id            0
dtype: int64
historical transaction set NA data authorized_flag         0
card_id                 0
city_id                 0
category_1              0
installments            0
category_3              0
merchant_category_id    0
merchant_id             0
month_lag               0
p

In [8]:
# a helper function to save memory usage, u don't need to understand this one
# @input dataframe
# @return dataframe
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)


### Feature Extraction

Manual feature engineering can be a tedious process (which is why we use automated feature engineering with featuretools!) and often relies on domain expertise. I will concentrate of getting as much info as possible into the final training dataframe. The idea is that the model will then pick up on which features are important rather than us having to decide that. Basically, our approach is to make as many features as possible and then give them all to the model to use! Later, we can perform feature reduction using the feature importances from the model or other techniques such as PCA.

In [ ]:
# print all current features
print(list(train))
print(list(test))
print(list(historical_transactions))
print(list(new_transactions))
print(list(merchants))

['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target']
['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3']
['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments', 'category_3', 'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date', 'category_2', 'state_id', 'subsector_id']
['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments', 'category_3', 'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date', 'category_2', 'state_id', 'subsector_id']
['merchant_id', 'merchant_group_id', 'merchant_category_id', 'subsector_id', 'numerical_1', 'numerical_2', 'category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3', 'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6', 'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12', 'category_4', 'city_id', 'state_id', 'catego

We need to understand the feature now.
1. the train dataset, we can see that it has card_id, this should be the unique id we can use for user profile, and group or aggregate the data w.r.t this feature. the feature_# in the data_dictionary descibed as Anonymized card categorical feature, we need to understand it later.
2. historical and new transactions. these two dataset have similar data structure.  
    a. 'purchase_date': max','min', require feature extraction.
        - 'month','hour','weekofyear','dayofweek','year', require no further process
        - 'weekend': 'sum', 'mean'
    b. 'merchant_id', 'merchant_category_id', 'subsector_id' need no more process.  
    c. 'purchase_amount': 'sum','max','min','mean','var'.  
    d. 'month_lag': 'max','min','mean','var'  
    e. 'month_diff': 'mean'  
    f. 'authorized_flag': 'sum, 'mean'  
    g. 'category_1': 'sum', 'mean'  
    h. 'category_2': 'mean'  
    i. 'category_3': 'mean'  
    j. 'card_id': 'size'

In [ ]:
# helper function, create new column
def create_column(prefix, aggs):
    return [prefix + '_' + key + '_' + agg for key in aggs.keys() for agg in aggs[key]]

In [ ]:
datetime.datetime.today()

datetime.datetime(2019, 4, 29, 3, 27, 46, 577163)

In [ ]:
## data preprocessing

# 1. One important feature here is purchase_date feature, we need to extract it into year, month,
# week of year, day of week, weekend, hour
# 2. get time difference to today which 
# 3. normalize binary data to 1/0 int


for df in [historical_transactions, new_transactions]:

    # date conversion
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df['purchase_date'].dt.dayofweek >= 5).astype(int) # 0-5 week day
    df['hour'] = df['purchase_date'].dt.hour

    ## time difference
    # https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

    # normalization
    # TODO still not well done here
    df['authorized_flag'] = df['authorized_flag'].map({'Y': 1, 'N': 0})
    df['category_1'] = df['category_1'].map({'Y': 1, 'N': 0})

In [ ]:
merchants['category_1'] = merchants['category_1'].map({'Y': 1, 'N': 0})
merchants['category_4'] = merchants['category_4'].map({'Y': 1, 'N': 0})

### Feature aggregation
`groupby`: group a dataframe by a column. In this case we will group by the `card_id`.  
`agg`: perform a calculation on the grouped data such as taking the `mean` of columns. We can either call the function directly (grouped_df.mean()) or use the agg function together with a list of transforms (`grouped_df.agg([mean, max, min, sum])`)  
`merge`: match the aggregated statistics to the appropriate client. We need to merge the original training data with the calculated stats on the `card_id` column.

#### aggregate merchant data (group_by merchant_id)

In [ ]:
def aggregate_transactions_merchants(df, prefix):
    
    agg_func = {
        'numerical_1': ['sum', 'mean'],
        'numerical_2': ['sum', 'mean'],
        'category_1': ['mean'],
        'category_4': ['mean'],
        'category_2': ['mean'],
        'most_recent_sales_range_A': ['mean'],
        'most_recent_sales_range_B': ['mean'],
        'most_recent_sales_range_C': ['mean'],
        'most_recent_sales_range_D': ['mean'],
        'most_recent_sales_range_E': ['mean'],
        'most_recent_purchases_range': ['nunique'],
        'merchant_group_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'avg_sales_lag3': ['sum','mean'],
        'avg_purchases_lag3': ['sum','mean'],
        'active_months_lag3': ['sum', 'mean'],
        'avg_sales_lag6': ['sum','mean'],
        'avg_purchases_lag6': ['sum','mean'],
        'active_months_lag6': ['sum', 'mean'],
        'avg_sales_lag12': ['sum','mean'],
        'avg_purchases_lag12': ['sum','mean'],
        'active_months_lag12': ['sum', 'mean'],
    }
    
    agg_df = df.groupby(['merchant_id']).agg(agg_func)
    agg_df.columns = [prefix + '_'.join(col).strip() for col in agg_df.columns.values]
    agg_df.reset_index(inplace=True)
    
    df = (df.groupby('merchant_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))
    
    agg_df = pd.merge(df, agg_df, on='merchant_id', how='left')
    
    return agg_df

In [ ]:
merchants = aggregate_transactions_merchants(merchants, prefix='merch_')
merchants = reduce_mem_usage(merchants)
gc.collect()

In [ ]:
# merge transaction data to hist transaction data, leaving this out for now since memory can't handle it
historical_transactions = pd.merge(historical_transactions, merchants, on='merchant_id', how='left')

In [ ]:
# merge transaction data to new transaction data
new_transactions = pd.merge(new_transactions, merchants, on='merchant_id', how='left')

In [ ]:
new_transactions.head()

#### aggregate new and hist transaction data (group by card_id)

In [ ]:
# create aggregation columns (features)
aggs = {}
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
    aggs[col] = ['nunique']

aggs['purchase_amount'] = ['sum','max','min','mean','var']
aggs['installments'] = ['sum','max','min','mean','var']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var']
aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# features from merchants data
aggs['merch_numerical_1_sum'] = ['sum', 'mean', 'max', 'min']
aggs['merch_numerical_1_mean'] = ['sum', 'mean', 'max', 'min']
aggs['merch_numerical_2_sum'] = ['sum', 'mean', 'max', 'min']
aggs['merch_numerical_2_mean'] = ['sum', 'mean', 'max', 'min']
aggs['merch_category_1_mean'] =  ['mean']
aggs['merch_category_4_mean'] = ['mean']
aggs['merch_category_2_mean'] = ['mean']
aggs['merch_most_recent_sales_range_A_mean'] = ['mean']
aggs['merch_most_recent_sales_range_B_mean'] = ['mean']
aggs['merch_most_recent_sales_range_C_mean'] = ['mean']
aggs['merch_most_recent_sales_range_D_mean'] = ['mean']
aggs['merch_most_recent_sales_range_E_mean'] = ['mean']
aggs['merch_most_recent_purchases_range_nunique'] = ['nunique']
aggs['merch_merchant_group_id_nunique'] = ['nunique']
aggs['merch_merchant_category_id_nunique'] = ['nunique']
aggs['merch_avg_sales_lag3_sum'] = ['sum','mean']
aggs['merch_avg_sales_lag3_mean'] = ['sum','mean']
aggs['merch_avg_purchases_lag3_sum'] = ['sum','mean']
aggs['merch_avg_purchases_lag3_mean'] = ['sum','mean']
aggs['merch_active_months_lag3_sum'] = ['sum', 'mean']
aggs['merch_active_months_lag3_mean'] = ['sum', 'mean']
aggs['merch_avg_sales_lag6_sum'] = ['sum','mean']
aggs['merch_avg_sales_lag6_mean'] = ['sum','mean']
aggs['merch_avg_purchases_lag6_sum'] = ['sum','mean']
aggs['merch_avg_purchases_lag6_mean'] = ['sum','mean']
aggs['merch_active_months_lag6_sum'] = ['sum', 'mean']
aggs['merch_active_months_lag6_mean'] = ['sum', 'mean']
aggs['merch_avg_sales_lag12_sum'] = ['sum','mean']
aggs['merch_avg_sales_lag12_mean'] = ['sum','mean']
aggs['merch_avg_purchases_lag12_sum'] = ['sum','mean']
aggs['merch_avg_purchases_lag12_mean'] = ['sum','mean']
aggs['merch_active_months_lag12_sum'] = ['sum', 'mean']
aggs['merch_active_months_lag12_mean'] = ['sum', 'mean']

aggs['category_1'] = ['sum', 'mean']

for col in ['category_2','category_3']:
    aggs[col+'_mean'] = ['mean'] 

In [ ]:
aggs

In [ ]:
# historical transactions data feature engineering
#
# append a column to historical transactions first
for col in ['category_2', 'category_3']:
     historical_transactions[col + '_mean'] = historical_transactions.groupby([col])['purchase_amount'].transform('mean')

# historical transaction features
hist_columns = create_column('hist', aggs)

hist_trans_agg = historical_transactions.groupby('card_id').agg(aggs)
hist_trans_agg.columns = hist_columns
hist_trans_agg.reset_index(drop=False, inplace=False)
hist_trans_agg['hist_purchase_date_diff'] = \
    (hist_trans_agg['hist_purchase_date_max'] - hist_trans_agg['hist_purchase_date_min']).dt.days
hist_trans_agg['hist_purchase_date_average'] = \
    (hist_trans_agg['hist_purchase_date_diff']) / hist_trans_agg['hist_card_id_size']
hist_trans_agg['hist_purchase_date_uptonow'] = \
    (datetime.datetime.today() - hist_trans_agg['hist_purchase_date_max']).dt.days

# merge into train and test dataset
train = train.merge(hist_trans_agg, on='card_id', how='left')
test = test.merge(hist_trans_agg, on='card_id', how='left')

# gc collection, save memory
del hist_trans_agg;gc.collect()
print('...done')

In [ ]:
# new transactions data feature engineering
#
# append a column to new transactions first
for col in ['category_2', 'category_3']:
     new_transactions[col + '_mean'] = new_transactions.groupby([col])['purchase_amount'].transform('mean')

# new transaction features
new_columns = create_column('new', aggs)

new_trans_agg = new_transactions.groupby('card_id').agg(aggs)
new_trans_agg.columns = new_columns
new_trans_agg.reset_index(drop=False, inplace=False)
new_trans_agg['new_purchase_date_diff'] = \
    (new_trans_agg['new_purchase_date_max'] - new_trans_agg['new_purchase_date_min']).dt.days
new_trans_agg['new_purchase_date_average'] = \
    (new_trans_agg['new_purchase_date_diff']) / new_trans_agg['new_card_id_size']
new_trans_agg['new_purchase_date_uptonow'] = \
    (datetime.datetime.today() - new_trans_agg['new_purchase_date_max']).dt.days

# merge into train and test dataset
train = train.merge(new_trans_agg, on='card_id', how='left')
test = test.merge(new_trans_agg, on='card_id', how='left')

# gc collection
del new_trans_agg;gc.collect()
print('...done')

In [ ]:
# gc collection
del historical_transactions;gc.collect()
del new_transactions;gc.collect()
print('...done')

In [ ]:
# outliers
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

In [ ]:
# train, test dataset feature engineering
for df in [train, test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['new_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_purchase_date_max',\
                     'new_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['hist_card_id_size'] + df['new_card_id_size']
    df['purchase_amount_total'] = df['new_purchase_amount_sum'] + df['hist_purchase_amount_sum']

for f in ['feature_1','feature_2','feature_3']:
    order_label = train.groupby([f])['outliers'].mean()
    train[f] = train[f].map(order_label)
    test[f] = test[f].map(order_label)

In [ ]:
train.head()

In [ ]:
# plot
plt.figure(figsize=(12,6))
plt.subplots_adjust(wspace=0.4, hspace=0.3)
plt.subplot(231)
plt.hist(train['feature_1'])
plt.xlabel('feature_1')
plt.ylabel('frequency')
plt.subplot(232)
plt.hist(train['feature_2'])
plt.xlabel('feature_2')
plt.ylabel('frequency')
plt.subplot(233)
plt.hist(train['feature_3'])
plt.xlabel('feature_3')
plt.ylabel('frequency')
plt.subplot(234)
plt.hist(train['target'])
plt.xlabel('target')
plt.ylabel('frequency')
plt.subplot(235)
ax = sns.lineplot(x = "month", y = "target", 
                  markers = True, dashes = False, data = train)
plt.xticks(rotation = 45)
ax.set_xlabel('Purchase Month')
plt.subplot(236)
ax = sns.lineplot(x = "weekofyear", y = "target", 
                  markers = True, dashes = False, data = train)
plt.xticks(rotation = 45)
ax.set_xlabel('week of year')

We can do the modeling now!

## Evaluation

In [ ]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import neighbors

import gc
import matplotlib.pyplot as plt


In [ ]:

train_columns = [c for c in train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = train['target']
del train['target']

In [ ]:
train.head()

### Let's try some models

In [ ]:
print(target.values)

In [ ]:
def lightgbm(train, test, n_folds=5):
    param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['outliers'].values)):
        print("fold {}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
        val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)

        num_round = 10000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
        oof[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = train_columns
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits

    valid_score = np.sqrt(mean_squared_error(oof, target))
    return feature_importance_df, valid_score

In [ ]:
feature_importance, valid_score = lightgbm(train, test, n_folds=5)

In [ ]:
print("the best validation RMSE score is {}".format(valid_score))

In [ ]:
# plot the features
sns.set(font_scale=1.5)
cols = (feature_importance[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:20].index)

best_features = feature_importance.loc[feature_importance.Feature.isin(cols)]

plt.figure(figsize=(10,15))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Top 20 Features (avg over folds)')
plt.tight_layout()
# plt.savefig('lgbm_importances.png')
